In [1362]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score



In [1363]:
data  = pd.read_csv('data/water_potability.csv')

data= data.dropna()

In [1364]:
notpotable  = data[data['Potability']==0]
potable = data[data['Potability']==1]  

from sklearn.utils import resample
df_minority_upsampled = resample(potable, replace = True, n_samples = 1200) 

from sklearn.utils import shuffle
data = pd.concat([notpotable, df_minority_upsampled])
data = shuffle(data) 

In [1365]:

y = data['Potability']
X = data.drop(columns=['Potability'], axis = 1)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=101)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)





In [1366]:
rf = RandomForestClassifier(n_estimators=100,
                            max_depth=50,
                            random_state=101,
                            max_features=9,
                            min_samples_leaf = 1,
                            n_jobs=-1)



rf.fit(X_train_scaled , y_train)
pred_train = rf.predict(X_train_scaled )
pred_test = rf.predict(X_test_scaled )

print(accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test))

1.0 0.8395833333333333
